In [3]:
# predict which clients are more likely to default on their loans

import sys
from pathlib import Path
import subprocess
import os
import gc
from glob import glob

import joblib
import numpy as np
import pandas as pd
import polars as pl
from datetime import datetime

from sklearn.metrics import roc_auc_score
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

ROOT = '/kaggle/input/home-credit-default-risk-model-stability'



In [4]:

def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    # calculate and display the memory usage of the dataframe
    start_mem = df.memory_usage().sum() / 1024**2
    print(f"Memory usage of dataframe is {start_mem:.2f} MB").format(start_mem)

    for col in df.columns:
        col_type = df[col].dtype

        if str(col_type)=="category":
            continue

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
class Pipeline:
    @staticmethod
    #Standardize the dtype
    def set_table_dtypes(df): 
        for col in df.columns:
            if col in ["case_id", "WEEK_NUM", "num_group1", "num_group2"]:
                df = df.with_columns(pl.col(col).cast(pl.Int64))
            elif col in ["date_decision"]:
                df = df.with_columns(pl.col(col).cast(pl.Date))
            elif col[-1] in ("P", "A"):
                df = df.with_columns(pl.col(col).cast(pl.Float64))
            elif col[-1] in ("M",):
                df = df.with_columns(pl.col(col).cast(pl.String))
            elif col[-1] in ("D",):
                df = df.with_columns(pl.col(col).cast(pl.Date))            

        return df
    
    @staticmethod
    #Change the feature for D to the difference in days from date_decision.
    def handle_dates(df): 
        for col in df.columns:
            if (col[-1] in ("D",)) and ('count' not in col):
                df = df.with_columns(pl.col(col) - pl.col("date_decision"))
                df = df.with_columns(pl.col(col).dt.total_days())
                
        df = df.drop("date_decision", "MONTH")

        return df
    
    @staticmethod
    #Remove those with an average is_null exceeding 0.95 and those that do not fall within the range 1 < nunique < 200.
    def filter_cols(df): 
        for col in df.columns:
            # if col in ["decision_month", "decision_weekday"]:
            #     df = df.drop(col)
            #     continue
            # if ('amtde' in col) or ('bureau_b2' in col): # for ohter option
            #     continue

            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].is_null().mean()
                if isnull > 0.95:
                    df = df.drop(col)

        for col in df.columns:
            # if '_depth2_' in col:
            #     continue
            if (col not in ["target", "case_id", "WEEK_NUM", ]) & (df[col].dtype == pl.String):
                freq = df[col].n_unique()

                if (freq == 1) | (freq > 50):#50 #len(df) * 0.20): # 95 # fe4 down at fq20
                    df = df.drop(col)
            
            # eliminate year, month feature
            # 644
            if (col[-1] not in ["P", "A", "L", "M"]) and (('month_' in col) or ('year_' in col)):# or ('num_group' in col):
            # if (('month_' in col) or ('year_' in col)):# or ('num_group' in col):
                df = df.drop(col)

        return df


In [ ]:
class Aggregator:
    @staticmethod
    def num_expr(df):
        cols = [col for col in df.columns if (col[-1] in ("T","L","M","D","P","A")) or ("num_group" in col)]

        expr_1 = [pl.max(col).alias(f"max_{col}") for col in cols]
        expr_2 = [pl.min(col).alias(f"min_{col}") for col in cols]

        cols2 = [col for col in df.columns if col[-1] in ("L", "A")]
        expr_3 = [pl.mean(col).alias(f"mean_{col}") for col in cols2] + [pl.std(col).alias(f"std_{col}") for col in cols2] + \
            [pl.sum(col).alias(f"sum_{col}") for col in cols2] + [pl.median(col).alias(f"median_{col}") for col in cols2] # + \
   
        return expr_1 + expr_2 + expr_3 # + [pl.col(col).diff().last().alias(f"diff-last_{col}") for col in cols3] # + expr_4
    
    @staticmethod
    def applprev2_exprs(df):
        cols = [col for col in df.columns if "num_group" not in col]
        expr_2 = [pl.first(col).alias(f"first_{col}") for col in cols]#  + [pl.last(col).alias(f"last_{col}") for col in cols]
        return []#expr_2

    @staticmethod
    def bureau_a1(df):
        cols = [col for col in df.columns if (col[-1] in ("T","L","M","D","P","A")) or ("num_group" in col)]
        expr_1 = [pl.max(col).alias(f"max_{col}") for col in cols]
        expr_2 = [pl.min(col).alias(f"min_{col}") for col in cols]

        cols2 = [
        'annualeffectiverate_199L', 'annualeffectiverate_63L',
        'contractsum_5085717L', 
        'credlmt_230A', 'credlmt_935A',
       'nominalrate_281L', 'nominalrate_498L',
       'numberofcontrsvalue_258L', 'numberofcontrsvalue_358L',
       'numberofinstls_229L', 'numberofinstls_320L',
       'numberofoutstandinstls_520L', 'numberofoutstandinstls_59L',
       'numberofoverdueinstlmax_1039L', 'numberofoverdueinstlmax_1151L',
       'numberofoverdueinstls_725L', 'numberofoverdueinstls_834L',
       ]

        expr_3 = [pl.mean(col).alias(f"mean_{col}") for col in cols2] + [pl.std(col).alias(f"std_{col}") for col in cols2] + \
            [pl.sum(col).alias(f"sum_{col}") for col in cols2] + [pl.median(col).alias(f"median_{col}") for col in cols2] + \
            [pl.first(col).alias(f"first_{col}") for col in cols2] # + [pl.last(col).alias(f"last_{col}") for col in cols2] # not applied
        
        return expr_1 + expr_2 + expr_3    


    @staticmethod
    def bureau_b1(df): 
        return []
    
    
    @staticmethod
    def bureau_b2(df):  
        return []


    @staticmethod
    def deposit_exprs(df):
        cols = [col for col in df.columns if (col[-1] in ("T","L","M","D","P","A")) or ("num_group" in col)]
        expr_1 = [pl.max(col).alias(f"max_{col}") for col in cols] + [pl.min(col).alias(f"min_{col}") for col in cols] # + \

        return expr_1 # + expr_2 #+ expr_ngmax

    @staticmethod
    def debitcard_exprs(df):
        # cols = [col for col in df.columns if (col[-1] in ["A"])]
        cols = [col for col in df.columns if (col[-1] in ("T","L","M","D","P","A")) or ("num_group" in col)]
        expr_1 = [pl.max(col).alias(f"max_{col}") for col in cols] + [pl.min(col).alias(f"min_{col}") for col in cols] 

        return expr_1 


    @staticmethod
    def person_expr(df):
        cols1 = ['empl_employedtotal_800L', 'empl_employedfrom_271D', 'empl_industry_691L', 
                 'familystate_447L', 'incometype_1044T', 'sex_738L', 'housetype_905L', 'housingtype_772L',
                 'isreference_387L', 'birth_259D', ]
        expr_1 = [pl.first(col).alias(f"first_{col}") for col in cols1]
        
        expr_2 = [pl.col("mainoccupationinc_384A").max().alias("mainoccupationinc_384A_max"), 
                  pl.col("mainoccupationinc_384A").filter(pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("mainoccupationinc_384A_any_selfemployed")]
        
        return expr_1 + expr_2
    
    @staticmethod
    def person_2_expr(df):
        # cols = [col for col in df.columns]
        cols = ['empls_economicalst_849M', 'empls_employedfrom_796D', 'empls_employer_name_740M']

        expr_1 = [pl.first(col).alias(f"first_{col}") for col in cols]
        expr_2 = [pl.last(col).alias(f"last_{col}") for col in cols]

        return expr_1 + expr_2 # + expr_3# + expr_ngc 

    @staticmethod
    def other_expr(df):
        expr_1 = [pl.first(col).alias(f"__other_{col}") for col in df.columns if ('num_group' not in col) and (col != 'case_id')]
        return expr_1
    
    
    @staticmethod
    def tax_a_exprs(df):
        cols = [col for col in df.columns if (col[-1] in ("T","L","M","D","P","A")) or ("num_group" in col)]
        expr_1 = [pl.max(col).alias(f"max_{col}") for col in cols] + [pl.min(col).alias(f"min_{col}") for col in cols] + \
            [pl.last(col).alias(f"last_{col}") for col in cols] + \
            [pl.first(col).alias(f"first_{col}") for col in cols] + \
            [pl.mean(col).alias(f"mean_{col}") for col in cols] + \
            [pl.std(col).alias(f"std_{col}") for col in cols]

        expr_4 = [pl.col(col).fill_null(strategy="zero").apply(lambda x: x.max() - x.min()).alias(f"max-min_gap_depth2_{col}") for col in ['amount_4527230A']]

        return expr_1 + expr_4


    @staticmethod
    def bureau_a2(df): 
        cols = [col for col in df.columns if (col[-1] in ("T","L","M","D","P","A")) or ("num_group" in col)]

        expr_1 = [pl.max(col).alias(f"max_depth2_{col}") for col in cols]
        expr_2 = [pl.min(col).alias(f"min_depth2_{col}") for col in cols]
        expr_3 = [pl.mean(col).alias(f"mean_depth2_{col}") for col in cols] + \
            [pl.std(col).alias(f"std_{col}") for col in cols]

        expr_4 = [pl.col(col).fill_null(strategy="zero").apply(lambda x: x.max() - x.min()).alias(f"max-min_gap_depth2_{col}") for col in ['collater_valueofguarantee_1124L', 'pmts_dpd_1073P', 'pmts_overdue_1140A',]]

        expr_ngc = [pl.count("num_group2").alias(f"count_depth2_a2_num_group2")]

        return expr_1 + expr_2 + expr_3 + expr_4 + expr_ngc 
    
    @staticmethod
    def get_exprs(df):
        exprs = Aggregator.num_expr(df)

        return exprs

In [ ]:
def agg_by_case(path, df):
    path = str(path)
    if '_applprev_1' in path:
        df = df.sort("num_group1").group_by("case_id").agg(Aggregator.get_exprs(df))

    elif '_credit_bureau_a_1' in path:
        df = df.sort("num_group1").group_by("case_id").agg(Aggregator.bureau_a1(df))

    elif '_credit_bureau_b_1' in path:
        df = df.sort("num_group1").group_by("case_id").agg(Aggregator.bureau_b1(df))

    elif '_deposit_1' in path:
        df = df.sort("num_group1").group_by("case_id").agg(Aggregator.deposit_exprs(df))
    elif '_debitcard_1' in path:
        df = df.sort("num_group1").group_by("case_id").agg(Aggregator.debitcard_exprs(df))
        
    elif '_tax_registry_a' in path:
        df = df.sort("num_group1").group_by("case_id").agg(Aggregator.tax_a_exprs(df))
    elif '_tax_registry_b' in path:
        df = df.sort("num_group1").group_by("case_id").agg(Aggregator.get_exprs(df))
    elif '_tax_registry_c' in path:
        df = df.sort("num_group1").group_by("case_id").agg(Aggregator.get_exprs(df))
        
    elif '_other_1' in path:
        df = df.sort("num_group1").group_by("case_id").agg(Aggregator.other_expr(df))
    elif '_person_1' in path:
        df = df.sort("num_group1").group_by("case_id").agg(Aggregator.person_expr(df))
    elif '_person_2' in path:
        df = df.group_by("case_id").agg(Aggregator.person_2_expr(df))

    elif '_credit_bureau_a_2' in path:
        df = df.group_by("case_id").agg(Aggregator.bureau_a2(df))
    elif '_credit_bureau_b_2' in path:
        df = df.group_by("case_id").agg(Aggregator.get_exprs(df))
    
    return df

def read_file(path, depth=None): 
    df = pl.read_parquet(path)
    df = df.pipe(Pipeline.set_table_dtypes)
    
    if depth in [1, 2]:
        df = agg_by_case(path, df)
    
    return df

def read_files(regex_path, depth=None):
    print(regex_path)
    chunks = []
    for path in glob(str(regex_path)):
        df = pl.read_parquet(path)
        df = df.pipe(Pipeline.set_table_dtypes)
        if depth in [1, 2]:
            df = agg_by_case(path, df)
        chunks.append(df)
        
    df = pl.concat(chunks, how="vertical_relaxed")
    df = df.unique(subset=["case_id"])
    
    return df

def feature_eng(df_base, depth_0, depth_1, depth_2):
    df_base = (
        df_base.with_columns(
            decision_month = pl.col("date_decision").dt.month(),
            decision_weekday = pl.col("date_decision").dt.weekday(),
        )
    )
        
    for i, df in enumerate(depth_0 + depth_1 + depth_2):
        df_base = df_base.join(df, how="left", on="case_id", suffix=f"_{i}")
        
    df_base = df_base.pipe(Pipeline.handle_dates)
    return df_base

def to_pandas(df_data, cat_cols=None):
    df_data = df_data.to_pandas()
    print(df_data.info())
    if cat_cols is None:
        cat_cols = list(df_data.select_dtypes("object").columns)
    
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    
    return df_data, cat_cols


In [ ]:
ROOT            = Path("/kaggle/input/home-credit-credit-risk-model-stability")

TRAIN_DIR       = ROOT / "parquet_files" / "train"
TEST_DIR        = ROOT / "parquet_files" / "test"

data_store = {
    "df_base": read_file(TEST_DIR / "test_base.parquet"),
    "depth_0": [
        read_file(TEST_DIR / "test_static_cb_0.parquet"),
        read_files(TEST_DIR / "test_static_0_*.parquet"),
    ],
    "depth_1": [
        read_files(TEST_DIR / "test_applprev_1_*.parquet", 1),
        read_files(TEST_DIR / "test_credit_bureau_a_1_*.parquet", 1),
        read_file(TEST_DIR / "test_credit_bureau_b_1.parquet", 1),
        read_file(TEST_DIR / "test_deposit_1.parquet", 1),
        read_file(TEST_DIR / "test_debitcard_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_a_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_b_1.parquet", 1),
        read_file(TEST_DIR / "test_tax_registry_c_1.parquet", 1),
        read_file(TEST_DIR / "test_person_1.parquet", 1),
        read_file(TEST_DIR / "test_other_1.parquet", 1),
    ],
    "depth_2": [
        read_files(TEST_DIR / "test_credit_bureau_a_2_*.parquet", 2),
        read_file(TEST_DIR / "test_credit_bureau_b_2.parquet", 2),
        read_file(TEST_DIR / "test_person_2.parquet", 2),
    ]
}

df_test = feature_eng(**data_store)
print("test data shape:\t", df_test.shape)

# load models
cat_cols = joblib.load('/kaggle/input/hclgb-models/cat_cols.pickle')
ls_models = glob(os.path.join('/kaggle/input/hclgb-models', "*.pkl"))
models = [joblib.load(fn) for fn in ls_models]

lgb_features = models[0].feature_name_

df_test = df_test.select(['case_id'] + lgb_features)

# print("train data shape:\t", df_train.shape)
print("test data shape:\t", df_test.shape)

del data_store
gc.collect()

df_test, _ = to_pandas(df_test, cat_cols)
df_test = reduce_mem_usage(df_test)

df_test.drop(columns=["case_id"]).to_csv('test_data.csv', index=False)
del df_test
gc.collect()

In [ ]:
ROOT = '/kaggle/input/home-credit-credit-risk-model-stability'

# load models
cat_cols = joblib.load('/kaggle/input/hclgb-models/cat_cols.pickle')
ls_models = glob(os.path.join('/kaggle/input/hclgb-models', "*.pkl"))
models = [joblib.load(fn) for fn in ls_models]

print(len(models), models)

lgb_features = models[0].feature_name_
len(lgb_features), lgb_features

def set_categoricals(df_data, cat_cols):
    df_data[cat_cols] = df_data[cat_cols].astype("category")
    return df_data

def lgb_prediction(feats, models):
    predictions = np.zeros(len(feats))
    for model in models:
        p = model.predict_proba(feats)[:, 1]
        predictions += p/len(models)
    return predictions

CHUNK_SIZE = 10 ** 6
reader = pd.read_csv('/kaggle/working/test_data.csv', chunksize=CHUNK_SIZE)
y_pred = []
for df_chunk in reader:
    # p = predictor.predict_proba(df_chunk).iloc[:, 1].values
    df_chunk = set_categoricals(df_chunk, cat_cols)
    p = lgb_prediction(df_chunk, models)
    y_pred.append(p)
    
y_pred = np.concatenate(y_pred, axis=0)

del reader
gc.collect()

df_subm = pd.read_csv(f"{ROOT}/sample_submission.csv")
df_subm = df_subm.set_index("case_id")

df_subm["score"] = y_pred#lgb_pred

df_subm.to_csv("submission.csv")
print(df_subm)